<a href="https://colab.research.google.com/github/NLP-END3/Session5/blob/main/Session5_END_YelpReviewFull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchtext.datasets import IMDB,YelpReviewFull,YahooAnswers

In [ ]:
help(IMDB), help(YelpReviewFull), help(YahooAnswers)

Help on function IMDB in module torchtext.datasets.imdb:

IMDB(root='.data', split=('train', 'test'))
    IMDB dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 25000
    
        test: 25000
    
    
    Number of classes
        2
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')

Help on function YelpReviewFull in module torchtext.datasets.yelpreviewfull:

YelpReviewFull(root='.data', split=('train', 'test'))
    YelpReviewFull dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 650000
    
        test: 50000
    
    
    Number of classes
        5
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Ca

(None, None, None)

In [ ]:
data_set = input('What is your choice of dataset? : IMDB, YelpReviewFull, YahooAnswers')

What is your choice of dataset? : IMDB, YelpReviewFull, YahooAnswersYelpReviewFull


In [ ]:
print(f'Chosen dataset : {data_set}')

Chosen dataset : YelpReviewFull


In [ ]:
#train_iter_IMDB = IMDB(split = 'train')
train_iter_YelpReviewFull = YelpReviewFull(split = 'train')
#train_iter_YahooAnswers = YahooAnswers(split = 'train')

## Selecting the right dataset

In [ ]:
#data_sets = {'IMDB':train_iter_IMDB, 'YelpReviewFull': train_iter_YelpReviewFull} #, 'YahooAnswers': train_iter_YahooAnswers}
train_iter = YelpReviewFull(split='train')

## Exploring the dataset

In [ ]:
len(train_iter), type(train_iter), next(iter(train_iter))

(650000,
 torchtext.data.datasets_utils._RawTextIterableDataset,
 (5,
  "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."))

## NUmber of positive and Negative examples

In [ ]:
train_iter = YelpReviewFull(split='train')

for (line_number, (label,text)) in enumerate(train_iter):
  if line_number < 5:
    print(label, text)

5 dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
2 Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients 

## Tokenization & Vocabulary

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
tokenizer = get_tokenizer('basic_english')

In [ ]:
tokenizer("I am an Indian since 1990")

['i', 'am', 'an', 'indian', 'since', '1990']

In [ ]:
train_iter = YelpReviewFull(split = 'train')

In [ ]:
len(train_iter)

650000

In [ ]:
def yield_tokens(data_iter):
  for _,text in data_iter:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter),specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [ ]:
len(vocab) # vocab size is 519819

519819

## Text and Label Pipeline

In [ ]:
text_pipeline = lambda x : vocab(tokenizer(x))
label_pipeline = lambda x : int(x) - 1

In [ ]:
text_pipeline("the movie is great")

[2, 1058, 14, 58]

In [ ]:
label_pipeline(5)

4

## Collate_fn

In [ ]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)  

In [ ]:
from torch.utils.data import DataLoader
train_iter = YelpReviewFull(split='train')
dataloader = DataLoader(train_iter, batch_size = 8, shuffle = False, collate_fn = collate_batch)

## Neural Network Model

In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
train_iter = YelpReviewFull(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
print(f'Vocabulary Size : {vocab_size}')
print(f'Embedding Size : {emsize}')
print(f'Number of Classes : {num_class}')

Vocabulary Size : 519819
Embedding Size : 64
Number of Classes : 5


In [ ]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # disuccees
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## Running the model

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = YelpReviewFull()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 9649 batches | accuracy    0.390
| epoch   1 |  1000/ 9649 batches | accuracy    0.488
| epoch   1 |  1500/ 9649 batches | accuracy    0.509
| epoch   1 |  2000/ 9649 batches | accuracy    0.530
| epoch   1 |  2500/ 9649 batches | accuracy    0.540
| epoch   1 |  3000/ 9649 batches | accuracy    0.541
| epoch   1 |  3500/ 9649 batches | accuracy    0.549
| epoch   1 |  4000/ 9649 batches | accuracy    0.551
| epoch   1 |  4500/ 9649 batches | accuracy    0.554
| epoch   1 |  5000/ 9649 batches | accuracy    0.558
| epoch   1 |  5500/ 9649 batches | accuracy    0.558
| epoch   1 |  6000/ 9649 batches | accuracy    0.562
| epoch   1 |  6500/ 9649 batches | accuracy    0.562
| epoch   1 |  7000/ 9649 batches | accuracy    0.561
| epoch   1 |  7500/ 9649 batches | accuracy    0.563
| epoch   1 |  8000/ 9649 batches | accuracy    0.565
| epoch   1 |  8500/ 9649 batches | accuracy    0.566
| epoch   1 |  9000/ 9649 batches | accuracy    0.566
| epoch   1 |  9500/ 9649 ba

## Testing Accuracy

In [ ]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.596
